In [102]:
!pip install git+https://github.com/huggingface/transformers accelerate
!pip install torch
!pip install qwen_vl_utils
!pip install torchvision
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /localscratch-nvme/3210831/pip-req-build-i9u65gf4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /localscratch-nvme/3210831/pip-req-build-i9u65gf4
  Resolved https://github.com/huggingface/transformers to commit e68ebb695f9d1d990462397e284e79d8729aafea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [103]:
import torch
from transformers import (
    Qwen2_5_VLForConditionalGeneration, AutoProcessor as QwenProcessor
)
from qwen_vl_utils import process_vision_info
from datasets import load_dataset
from PIL import Image

In [104]:
# Load Qwen 7B model
qwen_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto"
)
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs for Qwen")
    qwen_model = torch.nn.DataParallel(qwen_model)
qwen_model.to("cuda")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using 2 GPUs for Qwen


DataParallel(
  (module): Qwen2_5_VLForConditionalGeneration(
    (model): Qwen2_5_VLModel(
      (visual): Qwen2_5_VisionTransformerPretrainedModel(
        (patch_embed): Qwen2_5_VisionPatchEmbed(
          (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
        (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
        (blocks): ModuleList(
          (0-31): 32 x Qwen2_5_VLVisionBlock(
            (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
            (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
            (attn): Qwen2_5_VLVisionAttention(
              (qkv): Linear(in_features=1280, out_features=3840, bias=True)
              (proj): Linear(in_features=1280, out_features=1280, bias=True)
            )
            (mlp): Qwen2_5_VLMLP(
              (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
              (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
              (down_proj): Linear(in_features=34

In [105]:
qwen_processor = QwenProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

In [150]:
# Load VQAv2 data
data_vqa = load_dataset("HuggingFaceM4/VQAv2", split="validation[:1000]")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Repo card metadata block was not found. Setting CardData to empty.


In [151]:
def chat_with_qwen_vqav2_temp_03(data_point, agent_id="Qwen-0.3", round_num=1, own_previous=None, peer_solutions=None):
    image = data_point['image']
    if image is None:
        return "No image provided."

    question = data_point['question']

    own_previous_text = f"\nYour previous answer was: {own_previous}" if own_previous else ""

    context = ""
    if peer_solutions:
        for i, peer in enumerate(peer_solutions, 1):
            context += f"\nAgent {i} said: {peer.strip()}"

    prompt = f"""You are Agent {agent_id} in a multi-agent debate to solve the following commonsense reasoning problem:
Problem: {question}
{own_previous_text}
Here are the solutions from other agents:{context}
This is debate round {round_num}. Please carefully analyze all solutions—including your own—identify any flawed assumptions or logical inconsistencies, and provide your revised solution.
• If you believe your previous answer is correct, explain the logical reasoning and real-world knowledge supporting it.
• If you believe you made an error, explain the flawed assumption or inconsistency and provide a corrected solution.
• If you believe another agent’s answer is correct, explain why their reasoning aligns with commonsense knowledge.
You must respond in this exact format and include only ONE final answer:

Reason: [Your reasoning in 2–4 sentences.]

Answer: {{your final one-word answer here, in lowercase, inside double curly braces. For example: {{down}}}}

Do not include multiple answers or repeat the Reason/Answer sections. Only one Reason and one Answer should be in your response.

"""

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt}
            ],
        }
    ]

    text = qwen_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

    generated_ids = qwen_model.module.generate(**inputs, max_new_tokens=4096, temperature=0.3)
    trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], generated_ids)]
    output_text = qwen_processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
    
    return output_text.strip()


In [152]:
def chat_with_qwen_vqav2_temp_07(data_point, agent_id="Qwen-0.7", round_num=1, own_previous=None, peer_solutions=None):
    image = data_point['image']
    if image is None:
        return "No image provided."

    question = data_point['question']

    own_previous_text = f"\nYour previous answer was: {own_previous}" if own_previous else ""

    context = ""
    if peer_solutions:
        for i, peer in enumerate(peer_solutions, 1):
            context += f"\nAgent {i} said: {peer.strip()}"

    prompt = f"""You are Agent {agent_id} in a multi-agent debate to solve the following commonsense reasoning problem:
Problem: {question}
{own_previous_text}
Here are the solutions from other agents:{context}
This is debate round {round_num}. Please carefully analyze all solutions—including your own—identify any flawed assumptions or logical inconsistencies, and provide your revised solution.
• If you believe your previous answer is correct, explain the logical reasoning and real-world knowledge supporting it.
• If you believe you made an error, explain the flawed assumption or inconsistency and provide a corrected solution.
• If you believe another agent’s answer is correct, explain why their reasoning aligns with commonsense knowledge.
You must respond in this exact format and include only ONE final answer:

Reason: [Your reasoning in 2–4 sentences.]

Answer: {{your final one-word answer here, in lowercase, inside double curly braces. For example: {{down}}}}

Do not include multiple answers or repeat the Reason/Answer sections. Only one Reason and one Answer should be in your response.

"""

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": prompt}
            ],
        }
    ]

    text = qwen_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

    generated_ids = qwen_model.module.generate(**inputs, max_new_tokens=4096, temperature=0.7)
    trimmed = [out[len(inp):] for inp, out in zip(inputs["input_ids"], generated_ids)]
    output_text = qwen_processor.batch_decode(trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
    
    return output_text.strip()


In [153]:
a1 = chat_with_qwen_vqav2_temp_07(data_vqa[0])

In [154]:
a2 = chat_with_qwen_vqav2_temp_03(data_vqa[0])

In [155]:
print(a1,a2)

Reason: The person appears to be focused on the skateboard trick, looking down at the board to maintain balance and control during the jump.常识中，当一个人执行动作尤其是滑板技巧时，注意力通常集中在他们自己的动作上。

Answer: {down} Reason: The skateboarder appears to be looking down at his skateboard as he performs the trick. This is consistent with common skateboarding behavior where riders focus on their board during maneuvers to maintain control and balance.

Answer: {down}


In [156]:
#Extracting answer from boxed format
import re
answer1 = re.findall(r'\{(.*?)\}', a1)
answer2 = re.findall(r'\{(.*?)\}', a2)

In [157]:
print(answer1, answer2)
print(answer1[0], answer2[0])

['down'] ['down']
down down


In [128]:
qwen_03_correct = 0
qwen_07_correct = 0

In [129]:
for i in data_vqa:
    a1 = chat_with_qwen_vqav2_temp_03(i)
    a2 = chat_with_qwen_vqav2_temp_07(i)
    if re.findall(r'\{(.*?)\}', a1)[0] == i['multiple_choice_answer']:
        qwen_03_correct+=1
    if re.findall(r'\{(.*?)\}', a2)[0] == i['multiple_choice_answer']:
        qwen_07_correct+=1

In [130]:
print(qwen_03_correct, qwen_07_correct)

563 532


In [131]:
qwen_03_correct = 0
qwen_07_correct = 0

In [132]:
for i in data_vqa:
    a1 = chat_with_qwen_vqav2_temp_03(i)
    a2 = chat_with_qwen_vqav2_temp_07(i)
    if re.findall(r'\{(.*?)\}', a1)[0] == i['multiple_choice_answer']:
        qwen_03_correct+=1
    if re.findall(r'\{(.*?)\}', a2)[0] == i['multiple_choice_answer']:
        qwen_07_correct+=1

In [133]:
print(qwen_03_correct, qwen_07_correct)

559 532


DEBATE

In [158]:
history_q07 = []
history_q03 = []

q07_prev = None
q03_prev = None

debate_log = []

In [159]:
import json
import re
from tqdm import tqdm

output_path = "debate_results_for_qwen.jsonl"
max_rounds = 3

with open(output_path, "a", encoding="utf-8") as outfile:
    for data_point in tqdm(data_vqa, desc="Running debates", total=len(data_vqa)):
        try:
            history_q07 = []
            history_q03 = []
            debate_log = []

            q07_prev = None
            q03_prev = None

            final_answer = None
            consensus = False
            rounds_done = 0 

            #Round 0 has no peer context
            q07_response = chat_with_qwen_vqav2_temp_07(
                data_point=data_point,
                agent_id="Qwen-0.7",
                round_num=0,
                own_previous=None,
                peer_solutions=None
            )
            q03_response = chat_with_qwen_vqav2_temp_03(
                data_point=data_point,
                agent_id="Qwen-0.3",
                round_num=0,
                own_previous=None,
                peer_solutions=None
            )

            history_q07.append(q07_response)
            history_q03.append(q03_response)

            debate_log.append({
                "round": 0,
                "Qwen-0.7": q07_response,
                "Qwen-0.3": q03_response,
            })

            answer1 = re.findall(r'\{(.*?)\}', q07_response)
            answer2 = re.findall(r'\{(.*?)\}', q03_response)
            # print(answer1, answer2)
            final1 = answer1[0].strip().lower() if answer1 else None
            final2 = answer2[0].strip().lower() if answer2 else None
            # print(final1, final2)

            if final1 and final1 == final2:
                consensus = True
                final_answer = final1
                rounds_done = 0 
            else:
                # Rounds 1-3 have context (peer answers)
                q07_prev = q07_response
                q03_prev = q03_response

                for round_num in range(1, max_rounds + 1):
                    rounds_done = round_num

                    q07_response = chat_with_qwen_vqav2_temp_07(
                        data_point=data_point,
                        agent_id="Qwen-0.7",
                        round_num=round_num,
                        own_previous=q07_prev,
                        peer_solutions=[q03_prev]
                    )
                    history_q07.append(q07_response)

                    q03_response = chat_with_qwen_vqav2_temp_03(
                        data_point=data_point,
                        agent_id="Qwen-0.3",
                        round_num=round_num,
                        own_previous=q03_prev,
                        peer_solutions=[q07_response]
                    )
                    history_q03.append(q03_response)

                    debate_log.append({
                        "round": round_num,
                        "Qwen-0.7": q07_response,
                        "Qwen-0.3": q03_response,
                    })

                    q07_prev = q07_response
                    q03_prev = q03_response

                    answer1 = re.findall(r'\{(.*?)\}', q07_response)
                    answer2 = re.findall(r'\{(.*?)\}', q03_response)
                    # print(answer1, answer2, rounds_done)

                    final1 = answer1[0].strip().lower() if answer1 else None
                    final2 = answer2[0].strip().lower() if answer2 else None

                    if final1 and final1 == final2:
                        consensus = True
                        final_answer = final1
                        break  

            final_output = {
                "question_id": data_point["question_id"],
                "multiple_choice_answer": data_point["multiple_choice_answer"],
                "question": data_point["question"],
                "image_id": data_point.get("image_id", None),
                "answer": final_answer,
                "rounds": rounds_done,
                "consensus": consensus,
                "history": debate_log
            }

            outfile.write(json.dumps(final_output) + "\n")

        except Exception as e:
            print(f"Error processing question_id {data_point.get('question_id', 'unknown')}: {e}")


Running debates: 100%|██████████| 1000/1000 [1:27:41<00:00,  5.26s/it]


In [162]:
import json

data = []
with open('debate_results_for_qwen.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))


METRICS

In [168]:
consensus_correct = 0
num_consensus = 0

In [169]:
for i in data:
    if i['consensus']==True:
        num_consensus+=1
        if i['answer'] == i['multiple_choice_answer']:
            consensus_correct+=1

In [170]:
print(num_consensus, consensus_correct)

981 548


981/1000 consensus
548/1000 consensus are correct